# Feature 
1)Identify the reference location in the feature file for +/- strand 

2)Extract "CG"-center pattern rows from the original feature file

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [54]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, StratifiedKFold

input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/'

In [55]:
names = ['chrom', 'site_pos', 'align_strand', 'loc_in_ref', 'read_id', 'read_strand',
         'kmer_seq', 'kmer_signal_mean', 'kmer_signal_std', 'kmer_signal_length', 'kmer_signal_range',
         'label']
####site_pos: start position in 0-based,
##if alignd_strand = +, the pos is the C or G location in the reference, pos = [site_pos, site_pos + 1]
##if alignd_strand = -, the pos is the G or C location in the reference (C in the kmer still), pos = [site_pos, site_pos + 1]
##Filter is in need to pick up the CG pattern
df=pd.read_csv(os.path.join(input_path,'feature.test.csv'),sep='\t', header = None)
df.columns = names
df.head(3)

,chrom,site_pos,align_strand,loc_in_ref,read_id,read_strand,kmer_seq,kmer_signal_mean,kmer_signal_std,kmer_signal_length,kmer_signal_range,label
0,J02459.1,42108,-,6393,17ff58dc-512e-4b3a-abb8-5ab1bdb42f6a,t,CGGTGATACTTCGTCGC,"-1.099168,-1.107496,-1.508581,0.643744,-1.4208...","0.0,0.152239,0.111529,0.187366,0.141536,0.1590...","1,6,9,13,28,6,38,18,21,3,7,17,21,14,14,10,20","0.0,0.424678,0.399697,0.736942,0.549584,0.4496...",0
1,J02459.1,42105,-,6396,17ff58dc-512e-4b3a-abb8-5ab1bdb42f6a,t,TGATACTTCGTCGCTGT,"0.643744,-1.4208,-1.009653,1.055123,-0.112415,...","0.187366,0.141536,0.159047,0.5128,0.257095,0.1...","13,28,6,38,18,21,3,7,17,21,14,14,10,20,15,11,16","0.736942,0.549584,0.44966,2.473129,1.13664,0.5...",0
2,J02459.1,42104,-,6397,17ff58dc-512e-4b3a-abb8-5ab1bdb42f6a,t,GATACTTCGTCGCTGTC,"-1.4208,-1.009653,1.055123,-0.112415,-0.446686...","0.141536,0.159047,0.5128,0.257095,0.15307,0.25...","28,6,38,18,21,3,7,17,21,14,14,10,20,15,11,16,8","0.549584,0.44966,2.473129,1.13664,0.549585,0.5...",0


In [44]:
df[df.columns[-1]].value_counts()

1    1200
0    1000
2     300
Name: label, dtype: int64

## Forward/reversed exploration in reference

In [4]:
df_forward = df[df['align_strand']=='+']
df_forward.head(3)

,chrom,site_pos,align_strand,loc_in_ref,read_id,read_strand,kmer_seq,kmer_signal_mean,kmer_signal_std,kmer_signal_length,kmer_signal_range,label
981,J02459.1,32506,+,32506,dea70d60-bb7f-4227-aaf8-f08ebf58ac31,t,ACAGGATTCATTGTCCT,"-0.139989,0.814478,-0.123791,-1.037187,-2.0065...","0.197427,0.130198,0.20086,0.072271,0.076593,0....","3,9,22,4,3,22,3,4,3,8,13,14,19,10,3,10,21","0.47087,0.394512,1.005372,0.190893,0.178167,0....",0
982,J02459.1,32510,+,32510,dea70d60-bb7f-4227-aaf8-f08ebf58ac31,t,GATTCATTGTCCTGCTC,"-2.006501,-1.994546,0.57268,0.855838,0.182409,...","0.076593,0.126111,0.1171,0.36965,0.073719,0.31...","3,22,3,4,3,8,13,14,19,10,3,10,21,15,3,17,18","0.178167,0.534502,0.254525,0.94174,0.178167,1....",0
983,J02459.1,32512,+,32512,dea70d60-bb7f-4227-aaf8-f08ebf58ac31,t,TTCATTGTCCTGCTCAA,"0.57268,0.855838,0.182409,-1.078548,0.509049,0...","0.1171,0.36965,0.073719,0.312555,0.196774,0.15...","3,4,3,8,13,14,19,10,3,10,21,15,3,17,18,35,8","0.254525,0.94174,0.178167,1.069003,0.789026,0....",0


In [9]:
import statsmodels as sm
from cytosine.utils.fast5_helper import fast5_rescale, fast5_normalize_signal, raw_fast5

input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/test'
fast5_fn='00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5'

signal_group='Raw/Reads'
corrected_group='RawGenomeCorrected_001' #tombo resquiggle save location
basecall_group='Basecall_1D_001' #has to be save in the tombo requiggle step
basecall_subgroup='BaseCalled_template' #Attention: the basecall_subgroup can be 'basecalled_compl

# Test fast5 class
path = os.path.join(input_path, fast5_fn)
fast5 = raw_fast5(path, corrected_group, basecall_group, basecall_subgroup, signal_group)
# Extract signal
readid, fast5_signal = fast5.fast5_signal() #get readid

#Extract event information: #Raw signal --> Normalization
event = fast5.fast5_event()
raw_signal = fast5_rescale(path, fast5_signal)
norm_signal = fast5_normalize_signal(raw_signal,  method='mad')

In [33]:
basecalled_seq, raw_signal_list, norm_signal_list = "", [], []
for e in event:
#    print(e)
    basecalled_seq += e[2]
#    print(norm_signal[e[0]:(e[0] + e[1])])
    norm_signal_list.append(norm_signal[e[0]:(e[0] + e[1])]) #event start position: end position(start+length)
    raw_signal_list.append(raw_signal[e[0]:(e[0] + e[1])])
    assert len(norm_signal_list) == len(raw_signal_list)

In [34]:
motif_seq = 'CG'
motif = set(motif_seq)
seq_length = len(basecalled_seq)
seq_length

32675

In [36]:
basecalled_seq[1:3]

'TT'

In [ ]:
for i in range(0, seq_length):
    if seq[i:i + motif_length] in motif:
        mod_site_loc.append(i + mod_loc)
    else:
        continue
return mod_site_loc

In [100]:
kmer = 17
num_bases = (kmer - 1) // 2
num_bases

8

In [104]:
from pyfaidx import Fasta
ref_path = '/pod/2/li-lab/Ziwei/Nanopore/data/reference/Lambda_phage.fa'
##Original Start attributes are 1-based, End attributes are 0-based
##when one_based_attributes=False, it becomes 0-based
ref_0based = Fasta(ref_path,"fasta", one_based_attributes=False)
lambda_seq = ref_0based[0][32506-8:32506+8+1]
lambda_seq

>J02459.1:32498-32515
ACAGGATTCATTGTCCT

In [129]:
lambda_seq = ref_0based[0][32510:32510+1]
lambda_seq

>J02459.1:32510-32511
G

In [10]:
df_reverse = df[df['align_strand']=='-']
df_reverse.head(3)

,chrom,site_pos,align_strand,loc_in_ref,read_id,read_strand,kmer_seq,kmer_signal_mean,kmer_signal_std,kmer_signal_length,kmer_signal_range,label
0,J02459.1,42108,-,6393,17ff58dc-512e-4b3a-abb8-5ab1bdb42f6a,t,CGGTGATACTTCGTCGC,"-1.099168,-1.107496,-1.508581,0.643744,-1.4208...","0.0,0.152239,0.111529,0.187366,0.141536,0.1590...","1,6,9,13,28,6,38,18,21,3,7,17,21,14,14,10,20","0.0,0.424678,0.399697,0.736942,0.549584,0.4496...",0
1,J02459.1,42105,-,6396,17ff58dc-512e-4b3a-abb8-5ab1bdb42f6a,t,TGATACTTCGTCGCTGT,"0.643744,-1.4208,-1.009653,1.055123,-0.112415,...","0.187366,0.141536,0.159047,0.5128,0.257095,0.1...","13,28,6,38,18,21,3,7,17,21,14,14,10,20,15,11,16","0.736942,0.549584,0.44966,2.473129,1.13664,0.5...",0
2,J02459.1,42104,-,6397,17ff58dc-512e-4b3a-abb8-5ab1bdb42f6a,t,GATACTTCGTCGCTGTC,"-1.4208,-1.009653,1.055123,-0.112415,-0.446686...","0.141536,0.159047,0.5128,0.257095,0.15307,0.25...","28,6,38,18,21,3,7,17,21,14,14,10,20,15,11,16,8","0.549584,0.44966,2.473129,1.13664,0.549585,0.5...",0


In [16]:
######Reverse strand
from pyfaidx import Fasta
ref_path = '/pod/2/li-lab/Ziwei/Nanopore/data/reference/Lambda_phage.fa'
##Original Start attributes are 1-based, End attributes are 0-based
##when one_based_attributes=False, it becomes 0-based
ref_0based = Fasta(ref_path,"fasta", one_based_attributes=False)

lambda_seq = ref_0based[0][42104-8:42104+8+1].reverse.complement   #Pay attention to the sequence here because of the reverse!!!!
lambda_seq

>J02459.1:42113-42096 (complement)
GATACTTCGTCGCTGTC

In [17]:
lambda_seq = ref_0based[0][42108-8:42108+8+1].reverse.complement   #Pay attention to the sequence here because of the reverse!!!!
lambda_seq

>J02459.1:42117-42100 (complement)
CGGTGATACTTCGTCGC

### Extract CG pattern

In [56]:
kmer = 17
num_bases = (kmer - 1) // 2
num_bases

8

In [57]:
CG_index = list()
for seq in df['kmer_seq']:
    if seq[num_bases:num_bases+2] == 'CG':
        CG_index.append(seq) 

In [58]:
df_selected = df[df['kmer_seq'].isin(CG_index)]
df.shape, df_selected.shape

((2500, 12), (321, 12))

In [62]:
df_selected.to_csv('/pod/2/li-lab/Ziwei/Nanopore/daily/2021-11-02/test.csv', header = None, index = None, sep='\t')

In [46]:
df_selected[df_selected.columns[-1]].value_counts()

1    182
0    115
2     24
Name: label, dtype: int64

# 5hmC

In [49]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, StratifiedKFold

input_path='/pod/2/li-lab/Ziwei/Nanopore/results/feature_Guppy5.0.11'

names = ['chrom', 'site_pos', 'align_strand', 'loc_in_ref', 'read_id', 'read_strand',
         'kmer_seq', 'kmer_signal_mean', 'kmer_signal_std', 'kmer_signal_length', 'kmer_signal_range',
         'label']
####site_pos: start position in 0-based,
##if alignd_strand = +, the pos is the C or G location in the reference, pos = [site_pos, site_pos + 1]
##if alignd_strand = -, the pos is the G or C location in the reference (C in the kmer still), pos = [site_pos, site_pos + 1]
##Filter is in need to pick up the CG pattern
df=pd.read_csv(os.path.join(input_path,'T4_5hmC.csv'),sep='\t', header = None)
df.columns = names
df.shape

(40609, 12)

In [50]:
CG_index = list()
for seq in df['kmer_seq']:
    if seq[num_bases:num_bases+2] == 'CG':
        CG_index.append(seq) 
df_selected = df[df['kmer_seq'].isin(CG_index)]
df.shape, df_selected.shape

((40609, 12), (2964, 12))

In [51]:
df_selected

,chrom,site_pos,align_strand,loc_in_ref,read_id,read_strand,kmer_seq,kmer_signal_mean,kmer_signal_std,kmer_signal_length,kmer_signal_range,label
17,KJ477685.1,51400,-,115415,844b1103-ccf8-4813-875a-daa19cee7fdf,t,AAATTTGACGAGATTAA,"0.448836,-0.463578,-1.002813,0.974263,0.918213...","0.159101,0.07795,0.385753,0.132284,0.089614,0....","26,10,6,3,17,3,8,3,6,11,54,3,11,6,9,16,9","0.792257,0.256949,1.156268,0.310479,0.299773,0...",2
23,KJ477685.1,51384,-,115431,844b1103-ccf8-4813-875a-daa19cee7fdf,t,AAGCTTCTCGTGAAATT,"-0.206986,-0.786905,-0.824376,-0.259625,0.0481...","0.061191,0.191942,0.068274,0.065507,0.057903,0...","9,12,3,8,4,7,9,3,16,8,9,12,6,3,6,3,4","0.182005,0.631665,0.149887,0.182005,0.160593,0...",2
34,KJ477685.1,51345,-,115470,844b1103-ccf8-4813-875a-daa19cee7fdf,t,TAAATGAACGATACACT,"1.33292,0.153455,-0.694564,-0.465719,1.508043,...","0.27463,0.049707,0.203307,0.196976,0.134748,0....","8,3,8,8,7,14,6,3,10,3,16,5,5,9,29,9,4","0.845789,0.117769,0.738726,0.717315,0.406835,0...",2
43,KJ477685.1,51324,-,115491,844b1103-ccf8-4813-875a-daa19cee7fdf,t,TTTCAGAGCGTATTGAA,"0.302244,0.455425,0.601076,0.467503,-0.831514,...","0.126688,0.051365,0.083604,0.218597,0.18385,0....","13,13,7,3,3,11,3,10,7,17,4,5,11,3,11,3,16","0.438954,0.192712,0.267655,0.492484,0.44966,0....",2
54,KJ477685.1,51291,-,115524,844b1103-ccf8-4813-875a-daa19cee7fdf,t,GAGCTGAACGCCAGAAC,"-0.903781,-1.044798,-0.970694,0.378285,0.56385...","0.083784,0.088137,0.005047,0.181234,0.039418,0...","12,17,3,6,3,9,5,7,13,8,9,3,3,10,5,11,21","0.235537,0.353304,0.010706,0.524603,0.096356,0...",2
...,...,...,...,...,...,...,...,...,...,...,...,...
40507,KJ477685.1,111709,+,111709,9ce4d4d1-7244-4f84-962f-497610ca9c4d,t,GTTTATTACGTATCAGG,"-2.082215,0.499122,0.537139,0.802998,-0.647028...","0.294124,0.066086,0.07064,0.175142,0.353107,0....","31,3,11,19,28,3,3,13,3,6,12,6,5,12,3,21,25","1.470388,0.161877,0.229326,0.782408,1.605285,0...",2
40517,KJ477685.1,111730,+,111730,9ce4d4d1-7244-4f84-962f-497610ca9c4d,t,GTTGATACCGAAACCCG,"-1.839109,1.136997,0.894823,-1.037028,-1.27103...","0.374583,0.131919,0.139467,0.179526,0.14623,0....","12,14,3,8,9,26,17,3,4,10,14,23,3,3,5,36,3","1.510857,0.485632,0.323755,0.607041,0.364224,1...",2
40521,KJ477685.1,111737,+,111737,9ce4d4d1-7244-4f84-962f-497610ca9c4d,t,CCGAAACCCGCAATATA,"0.215837,0.155133,-0.504518,-0.928869,-1.45983...","0.108479,0.121595,0.135598,0.106568,0.091575,0...","3,4,10,14,23,3,3,5,36,3,10,14,6,4,5,11,22","0.256306,0.296775,0.404694,0.418184,0.377714,0...",2
40555,KJ477685.1,6714,-,160101,b8c993dd-cc97-4881-96c1-b48d06d75df9,t,TTGTACAACGTGGTATT,"1.167132,0.82658,-1.27394,0.899921,-0.194929,1...","0.180944,0.534603,0.224437,0.216369,0.347251,0...","36,3,23,11,23,9,10,3,3,5,22,16,3,7,7,3,16","0.833193,1.309304,0.912545,0.694328,1.269628,0...",2


In [55]:
df_feature = df[df.columns.drop(df.columns[-1])]
df_class = np.squeeze(df[df.columns[-1]].values) ##Convert the label into 1d-array

In [57]:
df_feature.shape, df_class.shape

((2500, 11), (2500,))

In [58]:
#Split the dataset into training, validation and test :
X = df_feature
y = df_class

#Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)

In [59]:
X_train.shape, X_test.shape

((2000, 11), (500, 11))

In [67]:
motif = set('CG')

motif_length = len(list(motif)[0])
motif_length

1

### Extract CG site only

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.6.1


In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [12]:
print(x_train.shape)

(60000, 28, 28)
